## LLM based image understanding 
#### Contributor(s): Utkarsh Pratiush <utkarshp1161@gmail.com> - 20th April 2025
#### edited - 
   

In [ ]:
import yaml
from stemOrchestrator.MLlayer.MLlayerLLM import generate_stem_analysis

# Load the YAML file
with open("credentials.yaml", "r") as f:
    cred = yaml.safe_load(f)

GEMINI_API_KEY = cred["GEMINI-API-KEY"]

In [ ]:


file_path = "/Users/utkarshpratiush/project/pyAutoMic/TEM/stemOrchestrator/data/drift_frame17.tiff"
prompt = "This is a scanning transmission electron HAADF image of SrTiO3 sample. Guess the direction of view.. also return an image making a square on it to highlight the unit cell"

response_text = generate_stem_analysis(file_path, prompt, api_key=GEMINI_API_KEY)
print(response_text)
